In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

In [2]:
DEV_SAMPLE_PERCENTAGE = 0.1
POSITIVE_DATA_FILE = "./data/rt-polaritydata/rt-polarity.pos"
NEGATIVE_DATA_FILE = "./data/rt-polaritydata/rt-polarity.neg"

#model params
EMBEDDING_DIM = 128
FILTER_ROWS = [3,4,5,6]
NUM_FILTERS = 64
DROPOUT_KEEP_PROB = 0.5
L2_REG_LAMBDA = 0.0005

#trainning params
BATCH_SIZE = 2048
NUM_EPOCH = 300
EVAL_STEPS = 100
CHECKPOINT_STEPS = 200
MAX_CHECK_POINTS = 3

In [3]:
# Data Preparation
# ==================================================

# Load data
print("Loading data...")
x_text, y = data_helpers.load_data_and_labels(POSITIVE_DATA_FILE, NEGATIVE_DATA_FILE)

# Build vocabulary
# 一行数据最多的词汇数
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor( max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))
print("x_shape:",x.shape)
print("y_shape:",y.shape)

Loading data...
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
x_shape: (10662, 56)
y_shape: (10662, 2)


In [4]:
data_counts = len(y)

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(data_counts))

x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
# 数据集切分为两部分
dev_sample_index = -1 * int(DEV_SAMPLE_PERCENTAGE * float(data_counts))
x_train, x_val = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_val = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print("Num of Vocabulary: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train shape = ",x_train.shape)
print("Val shape = ",x_val.shape)

print("smaple x:",x_train[0:5])
print("smaple y:",y_train[0:5])

Num of Vocabulary: 18758
Train shape =  (9596, 56)
Val shape =  (1066, 56)
smaple x: [[ 4719    59   182    34   190   804     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [  129  7044   284   146    80     3  1116    58    84  1386   182  1968
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [  146     3   453    88    34     1   190  8338 18328    12  2320  1573
   3840  6569 16227   112  1543   246    17  1722  5117     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0

In [5]:
tf.reset_default_graph()
text_cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=EMBEDDING_DIM,
            filter_sizes=FILTER_ROWS,
            num_filters=NUM_FILTERS,
            l2_reg_lambda=L2_REG_LAMBDA)

# Output directory for models and summaries
# 定义输出路径
timestamp = str(int(time.time()))
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
print("Writing to {}\n".format(out_dir))

# Define Training procedure
global_step = tf.Variable(0, name="global_step", trainable=False)
optimizer = tf.train.AdamOptimizer(1e-3)
# 计算梯度
grads_and_vars = optimizer.compute_gradients(text_cnn.loss)
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

# 保存变量的梯度值
grad_summaries = []
for g, v in grads_and_vars:
    if g is not None:
        grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
        sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        grad_summaries.append(grad_hist_summary)
        grad_summaries.append(sparsity_summary)
        
grad_summaries_merged = tf.summary.merge(grad_summaries)

 # Summaries for loss and accuracy
loss_summary = tf.summary.scalar("loss", text_cnn.loss)
acc_summary = tf.summary.scalar("accuracy", text_cnn.accuracy)

# Train Summaries
train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
train_summary_dir = os.path.join(out_dir, "summaries", "train")

# val summaries
val_summary_op = tf.summary.merge([loss_summary, acc_summary])
val_summary_dir = os.path.join(out_dir, "summaries", "val")

# Checkpoint directory.
checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
checkpoint_prefix = os.path.join(checkpoint_dir, "model")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
# 保存模型，最多保存5个模型
saver = tf.train.Saver(tf.global_variables(), max_to_keep=MAX_CHECK_POINTS)

# Write vocabulary
vocab_processor.save(os.path.join(out_dir, "vocab"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Writing to F:\JupyterRoot\DL_Parctise\tensorflow_study\14.cnn text classification\runs\1560327157

Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary

In [6]:
def train_step(x_batch, y_batch,train_summary_op,writer=None):
    feed_dict = {
      text_cnn.input_x: x_batch,
      text_cnn.input_y: y_batch,
      text_cnn.dropout_keep_prob: DROPOUT_KEEP_PROB
    }
    _, step, summaries, loss, accuracy = sess.run(
        [train_op, global_step, train_summary_op, text_cnn.loss, text_cnn.accuracy],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    if (step%10==0):
        print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
    if writer:
        writer.add_summary(summaries, step)

def val_step(x_batch, y_batch, val_summary_op,writer=None):
    feed_dict = {
      text_cnn.input_x: x_batch,
      text_cnn.input_y: y_batch,
      text_cnn.dropout_keep_prob: 1.0
    }
    step, summaries, loss, accuracy = sess.run(
        [global_step, val_summary_op, text_cnn.loss, text_cnn.accuracy],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
    if writer:
        writer.add_summary(summaries, step)

In [7]:
session_conf = tf.ConfigProto(
      allow_soft_placement=False,
      log_device_placement=False)
with tf.Session(config=session_conf) as sess:
    with tf.summary.FileWriter(train_summary_dir, sess.graph) as train_summary_writer:
        with tf.summary.FileWriter(val_summary_dir, sess.graph) as val_summary_writer:
            sess.run(tf.global_variables_initializer())
            # Generate batches
            batches = data_helpers.batch_iter(
                list(zip(x_train, y_train)), BATCH_SIZE, NUM_EPOCH)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch,train_summary_op,train_summary_writer)
                current_step = tf.train.global_step(sess, global_step)
                # 测试
                if current_step % EVAL_STEPS == 0:
                    print("\nEvaluation:")
                    val_step(x_val, y_val,val_summary_op, writer=val_summary_writer)
                # 保存模型
                if current_step % CHECKPOINT_STEPS == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))

num_batches_per_epoch: 5
2019-06-12T16:12:43.867681: step 10, loss 1.72706, acc 0.522792
2019-06-12T16:12:45.972055: step 20, loss 1.4794, acc 0.578348
2019-06-12T16:12:48.092359: step 30, loss 1.35764, acc 0.567664
2019-06-12T16:12:50.199751: step 40, loss 1.17878, acc 0.601852
2019-06-12T16:12:52.328071: step 50, loss 1.03224, acc 0.621083
2019-06-12T16:12:54.451358: step 60, loss 0.905689, acc 0.643162
2019-06-12T16:12:56.568697: step 70, loss 0.847995, acc 0.671652
2019-06-12T16:12:58.683072: step 80, loss 0.763915, acc 0.67735
2019-06-12T16:13:00.818335: step 90, loss 0.75241, acc 0.685897
2019-06-12T16:13:02.967618: step 100, loss 0.676618, acc 0.700855

Evaluation:
2019-06-12T16:13:03.140139: step 100, loss 0.623103, acc 0.652908
2019-06-12T16:13:05.252513: step 110, loss 0.628261, acc 0.7151
2019-06-12T16:13:07.393816: step 120, loss 0.575601, acc 0.749288
2019-06-12T16:13:09.517529: step 130, loss 0.513088, acc 0.76567
2019-06-12T16:13:11.628872: step 140, loss 0.485832, acc 0

Saved model checkpoint to F:\JupyterRoot\DL_Parctise\tensorflow_study\14.cnn text classification\runs\1560327157\checkpoints\model-1000

2019-06-12T16:16:21.864735: step 1010, loss 0.00514673, acc 1
2019-06-12T16:16:24.032982: step 1020, loss 0.00456088, acc 1
2019-06-12T16:16:26.166339: step 1030, loss 0.00580251, acc 1
2019-06-12T16:16:28.335791: step 1040, loss 0.00550995, acc 0.999288
2019-06-12T16:16:30.496500: step 1050, loss 0.00460873, acc 0.999288
2019-06-12T16:16:32.645007: step 1060, loss 0.00483391, acc 0.998576
2019-06-12T16:16:34.791228: step 1070, loss 0.00522646, acc 1
2019-06-12T16:16:36.923164: step 1080, loss 0.00358192, acc 1
2019-06-12T16:16:39.065479: step 1090, loss 0.00907051, acc 0.998576
2019-06-12T16:16:41.232499: step 1100, loss 0.003951, acc 1

Evaluation:
2019-06-12T16:16:41.258430: step 1100, loss 1.11464, acc 0.74015
2019-06-12T16:16:43.389200: step 1110, loss 0.00442076, acc 0.999288
2019-06-12T16:16:45.521725: step 1120, loss 0.00588906, acc 0.997863
2